# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

We'll compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Open Files and Show Top 5 Rows

In [1]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Removing Headers From a List of Lists

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [3]:
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


##  Extracting Ask HN and Show HN Posts

Now that we've removed the headers from `hn`, we're ready to filter our data. Since we're only concerned with post titles beginning with `Ask HN` or `Show HN`, we'll create new lists of lists containing just the data for those titles.

To find the posts that begin with either `Ask HN` or `Show HN`, we'll use the string method `startswith`

Also, we wish to control for case, we can use the `lower` method, which returns a lowercase version of the starting string

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
# verify the number of posts for each category
print(len(ask_posts), len(show_posts), len(other_posts))

1744 1162 17194


In [5]:
print(ask_posts[:3])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']]


In [6]:
print(show_posts[:3])

[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05']]


##  Calculating the Average Number of Comments for Ask HN and Show HN Posts
Now, we can determine if ask posts or show posts receive more comments on average.
By looping the lists `ask_posts` and `show_posts` separately:


In [7]:
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)


14.038417431192661


In [8]:
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


We found that `ask posts` receives more comments on average than `show posts` i.e. 14 vs 10

Since `ask posts` are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if `ask posts` created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

## Finding the Number of Ask Posts and Comments by Hour Created

We'll use the `datetime` module to work with the data in the `created_at` column.

`datetime` syntax references can be found here:
https://strftime.org/


`created_at` column example in ask_posts: `11/29/2015 22:46`

its corresponding `datetime` string is ` "%m/%d/%Y %H:%M"`

In [9]:
import datetime as dt

result_list = []

for post in ask_posts:
    create_at = post[6]
    num_comments = int(post[4])
    result_list.append([create_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date, num_comments = row[0], row[1]
    date_dt = dt.datetime.strptime(date, "%m/%d/%Y %H:%M")
    hour = date_dt.hour
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments


In [10]:
print(counts_by_hour)

{9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}


In [11]:
print(comments_by_hour)

{9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}


## Calculating the Average Number of Comments for Ask HN Posts by Hour

Calculate the average number of comments per post for posts created during each hour of the day.

Let's calculate by the following:

- Initialized an empty list (of lists) and assigned it to `avg_by_hour`
- Iterated over the keys of `comments_by_hour` and appended to `avg_by_hour` a list with the following attributes:

  - The first element is the key from `comments_by_hour`.
  - The second element is the value corresponding to that key divided by `counts_by_hour`.

The result should be a list of lists in which the first element is the hour and the second element is the average number of comments per post. Assign the result to a variable named `avg_by_hour`

In [12]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

print(avg_by_hour)

[[9, 5.5777777777777775], [13, 14.741176470588234], [10, 13.440677966101696], [14, 13.233644859813085], [16, 16.796296296296298], [23, 7.985294117647059], [12, 9.41095890410959], [17, 11.46], [15, 38.5948275862069], [21, 16.009174311926607], [20, 21.525], [2, 23.810344827586206], [18, 13.20183486238532], [3, 7.796296296296297], [5, 10.08695652173913], [19, 10.8], [1, 11.383333333333333], [22, 6.746478873239437], [8, 10.25], [4, 7.170212765957447], [0, 8.127272727272727], [6, 9.022727272727273], [7, 7.852941176470588], [11, 11.051724137931034]]


##  Sorting and Printing Values From a List of Lists

Although we now have the results we need, this format makes it difficult to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.


In [13]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

[[5.5777777777777775, 9], [14.741176470588234, 13], [13.440677966101696, 10], [13.233644859813085, 14], [16.796296296296298, 16], [7.985294117647059, 23], [9.41095890410959, 12], [11.46, 17], [38.5948275862069, 15], [16.009174311926607, 21], [21.525, 20], [23.810344827586206, 2], [13.20183486238532, 18], [7.796296296296297, 3], [10.08695652173913, 5], [10.8, 19], [11.383333333333333, 1], [6.746478873239437, 22], [10.25, 8], [7.170212765957447, 4], [8.127272727272727, 0], [9.022727272727273, 6], [7.852941176470588, 7], [11.051724137931034, 11]]


Once we got the sorted list `sorted_swap`, we'd like to print in a neat format.

Use the `str.format()` method to print the hour and average in the following format: `15:00: 38.59 average comments per post`

its corresponding `datetime` string is `"%H:%M {:.2f} average comments per post" `

In [14]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

# print the string "Top 5 Hours for Ask Posts Comments".
for hr, avg in sorted_swap[:5]:
    time_str = str(avg)
    date_dt = dt.datetime.strptime(time_str, "%H")
    time_str = date_dt.strftime("%H:%M")
    print("{} {:.2f} average comments per post".format(time_str, hr))

15:00 38.59 average comments per post
02:00 23.81 average comments per post
20:00 21.52 average comments per post
16:00 16.80 average comments per post
21:00 16.01 average comments per post


During which hours should you create a post to have a higher chance of receiving comments? 

Refer back to the documentation for the data set, the time zone is `Eastern Time in the US`
So we convert the times to the time zone we live in. e.g. `Pacific Time in the US`

In [15]:
tz_diff = dt.timedelta(hours=3)
# print the string "Top 5 Hours for Ask Posts Comments". in my timezone PST
for hr, avg in sorted_swap[:5]:
    time_str = str(avg)
    date_dt = dt.datetime.strptime(time_str, "%H")
    pst_dt = date_dt - tz_diff
    time_str = pst_dt.strftime("%H:%M")
    print("{} PST, {:.2f} average comments per post".format(time_str, hr))

12:00 PST, 38.59 average comments per post
23:00 PST, 23.81 average comments per post
17:00 PST, 21.52 average comments per post
13:00 PST, 16.80 average comments per post
18:00 PST, 16.01 average comments per post


## Conclusion
In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as `ask post` and created between `12:00 - 13:00 PST`.

However, it should be noted that the data set we analyzed `excluded posts without any comments`. Given that, it's more accurate to say that of `the posts that received comments`, `ask posts` received more comments on average and ask posts created between `12:00 - 13:00 PST` received the most comments on average.